# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [43]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [44]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lebu,-37.6167,-73.6500,13.45,94,4,3.92,CL,1674456166
1,1,rikitea,-23.1203,-134.9692,26.17,75,0,6.95,PF,1674456167
2,2,kapaa,22.0752,-159.3190,22.28,91,6,2.74,US,1674456167
3,3,busselton,-33.6500,115.3333,31.76,34,0,6.72,AU,1674456168
4,4,mar del plata,-38.0023,-57.5575,16.72,89,0,3.13,AR,1674455907


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [45]:
%%capture --no-display

# Configure the map plot
map = city_data_df.hvplot.points(
    "Lng",  
    "Lat",
    geo=True,
    tiles="OSM", 
    size="Humidity",
    color = "City",
    frame_width = 800,
    frame_height = 600,
    scale = 1.7
)

# Display the map
map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [46]:
# Narrow down cities that fit criteria and drop any results with null values

#A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df['Max Temp']>21)&(city_data_df['Max Temp']<27), :]
#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[city_data_df['Wind Speed']<4.5, :]
#Zero cloudiness
city_data_df = city_data_df.loc[city_data_df['Cloudiness']==0, :]

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,hilo,19.7297,-155.0900,23.49,95,0,2.06,US,1674456120
21,21,cape town,-33.9258,18.4232,23.95,68,0,2.24,ZA,1674455996
41,41,hermanus,-34.4187,19.2345,23.22,72,0,0.82,ZA,1674456184
134,134,filingue,14.3521,3.3168,21.81,19,0,3.87,NE,1674456225
147,147,knysna,-34.0363,23.0471,26.62,58,0,1.02,ZA,1674456231
149,149,nanakuli,21.3906,-158.1547,24.99,70,0,1.54,US,1674456232
156,156,plettenberg bay,-34.0527,23.3716,24.42,65,0,1.81,ZA,1674455650
157,157,kruisfontein,-34.0033,24.7314,26.07,57,0,3.56,ZA,1674455663
159,159,cidreira,-30.1811,-50.2056,22.40,87,0,1.95,BR,1674456236
198,198,buin,-33.7333,-70.7500,22.08,61,0,0.31,CL,1674456252


### Step 3: Create a new DataFrame called `hotel_df`.

In [47]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity", "Hotel Name"]]
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
6,hilo,US,19.7297,-155.0900,95,
21,cape town,ZA,-33.9258,18.4232,68,
41,hermanus,ZA,-34.4187,19.2345,72,
134,filingue,NE,14.3521,3.3168,19,
147,knysna,ZA,-34.0363,23.0471,58,
149,nanakuli,US,21.3906,-158.1547,70,
156,plettenberg bay,ZA,-34.0527,23.3716,65,
157,kruisfontein,ZA,-34.0033,24.7314,57,
159,cidreira,BR,-30.1811,-50.2056,87,
198,buin,CL,-33.7333,-70.7500,61,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [48]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hilo - nearest hotel: Dolphin Bay Hotel
cape town - nearest hotel: Townhouse Hotel
hermanus - nearest hotel: Aloe guest house
filingue - nearest hotel: No hotel found
knysna - nearest hotel: Graywood Hotel
nanakuli - nearest hotel: Aulani, A Disney Resort & Spa
plettenberg bay - nearest hotel: Milkwood Manor
kruisfontein - nearest hotel: No hotel found
cidreira - nearest hotel: Hotel Castelo
buin - nearest hotel: Hotel Casa Real
dahod - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
moron - nearest hotel: La Esquina
vila velha - nearest hotel: Hotel Prainha
loei - nearest hotel: ภูเรือธารา
champerico - nearest hotel: Hotel y Restaurante El Submarino
bethlehem - nearest hotel: Hotel Portas da Amazônia
seybaplaya - nearest hotel: No hotel found
mercedes - nearest hotel: Gran Hotel Mercedes
general roca - nearest hotel: Hotel Austral


,City,Country,Lat,Lng,Humidity,Hotel Name
6,hilo,US,19.7297,-155.0900,95,Dolphin Bay Hotel
21,cape town,ZA,-33.9258,18.4232,68,Townhouse Hotel
41,hermanus,ZA,-34.4187,19.2345,72,Aloe guest house
134,filingue,NE,14.3521,3.3168,19,No hotel found
147,knysna,ZA,-34.0363,23.0471,58,Graywood Hotel
149,nanakuli,US,21.3906,-158.1547,70,"Aulani, A Disney Resort & Spa"
156,plettenberg bay,ZA,-34.0527,23.3716,65,Milkwood Manor
157,kruisfontein,ZA,-34.0033,24.7314,57,No hotel found
159,cidreira,BR,-30.1811,-50.2056,87,Hotel Castelo
198,buin,CL,-33.7333,-70.7500,61,Hotel Casa Real


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [49]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    scale = 3,
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)